# Notebook I - Data Preprocessing & LBP
Laurine Dargaud, June 2022 (Biometric Systems, DTU course)

**Topic: Face Morphing Attack Detection**

## Parameters

**Parameters to fill in**
- final dimensions in pixels of preprocessed face images (`_IMAGE_HEIGHT_`and `_IMAGE_WIDTH_`)
- two paths where we can find Bona Fine and Morphs raw data to preprocess  (`_BF_PATH_`and `_MORPH_PATH_`)
- names of the database and morphing algorithm (`_DATABASE_` and `_MORPHING_ALGO`).

**Pre-requisites**
- Preprocessed images will be saved in `_BF_TARGET_PATH` and `_MORPH_TARGET_PATH`. Change them if your directory structure is different, and make sure these paths point to existing empty directories.
- Make sure that `data/{_DATABASE_}-Processed/bona-fide-lpb-<color>comp-cropped` and `data/{_DATABASE_}-Processed/morphed-{_MORPHING_ALGO_}-lpb-<color>comp-cropped` repositories exist for each color channel (6 empty directories to create).

In [1]:
_IMAGE_HEIGHT_ = _IMAGE_WIDTH_ = 500

# Paths where we can find the raw data
_BF_PATH_ = 'data/HDA_Morphing_DB_subset/FRGC/bonafide/'
_MORPH_PATH_ = 'data/HDA_Morphing_DB_subset/FRGC/morphs_facemorpher/'

# Infos about the dataset
_DATABASE_ = 'FRGCsub'
_MORPHING_ALGO_ = 'facemorpher'

# Target paths auto-generation
_BF_TARGET_PATH_ = f'data/{_DATABASE_}-Processed/bona-fide/'
_MORPH_TARGET_PATH = f'data/{_DATABASE_}-Processed/morphed-{_MORPHING_ALGO_}/'

## Phase 1 - Detect, align and crop faces

We use DeepFace library for this purpose (https://github.com/serengil/deepface).

### Imports & Methods

In [2]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import cv2

from os import listdir
from os.path import isfile, join, isdir

import random
from tqdm import tqdm

from lpb import main as Lpb

def process_faces(aType, H = _IMAGE_HEIGHT_, W = _IMAGE_WIDTH_, detector_backend = 'dlib'):
    if aType == 'bf':
        path = _BF_PATH_
        target_path = _BF_TARGET_PATH_
    elif aType == 'morphed':
        path = _MORPH_PATH_
        target_path = _MORPH_TARGET_PATH
    else:
        return None
    filenames = [f for f in listdir(path) if isfile(join(path, f))]
    N = len(filenames)
    print(f"[{aType}] We found {N} images in '{path}'")
    for k in tqdm(range (N)):
        # read image
        img_path = path+filenames[k]
        # detect and align face with DeepFace
        img = DeepFace.detectFace(img_path = img_path, target_size = (H,W), detector_backend=detector_backend)
        img = img * 255
        # normalization
        img = cv2.normalize(img,  None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        # save processed picture
        cv2.imwrite(target_path+filenames[k], img[:, :, ::-1])

### MAIN to run

In [4]:
# load real pics
bf_filenames = [f for f in listdir(_BF_PATH_) if isfile(join(_BF_PATH_, f))]
bf_len = len(bf_filenames)
print(f"[Bona Fide] We found {bf_len} images in '{_BF_PATH_}'")

# load morphed pics
m_filenames = [f for f in listdir(_MORPH_PATH_) if isfile(join(_MORPH_PATH_, f))]
m_len = len(m_filenames)
print(f"[Morphed faces] We found {m_len} images in '{_MORPH_PATH_}'")

[Morphed faces] We found 964 images in 'data/HDA_Morphing_DB_subset/FRGC/morphs_facemorpher/'


In [5]:
# detect, align and crop BF faces
print('== DETECTING FACE & CROPPING FOR BONA FIDE IMAGES ==')
process_faces('bf')

# detect, align and crop Morphed faces
print('== DETECTING FACE & CROPPING FOR MORPHED IMAGES ==')
process_faces('morphed')

== DETECTING FACE & CROPPING FOR MORPHED IMAGES ==
[morphed] We found 964 images in 'data/HDA_Morphing_DB_subset/FRGC/morphs_facemorpher/'


100%|██████████| 964/964 [12:17<00:00,  1.31it/s]


## Phase 2 - Generate LPB features for R, G and B color channels

### Imports & Methods

In [6]:
def apply_transformation(filenames, transformationFct, target_path = None, wantReturn = False):
    result = []
    all_processed_files = [f for f in listdir(target_path) if isfile(join(target_path, f))]
    for anImagePath in tqdm(filenames):
        filename = anImagePath.split('/')[-1]
        if filename not in all_processed_files:
            transformed_img = transformationFct(anImagePath)
            if wantReturn:
                result.append(transformed_img)
            else:
                # save transformed image
                cv2.imwrite(target_path+filename, transformed_img)
    if wantReturn:
        return result

def get_LPB(aFileName, colorTransformation=None, channel=None):
    img = cv2.imread(aFileName)
    if (img.shape[0] != _IMAGE_HEIGHT_) or (img.shape[1] != _IMAGE_WIDTH_):
        img = cv2.resize(img, (_IMAGE_HEIGHT_,_IMAGE_WIDTH_))
    if colorTransformation != None:
        img = cv2.cvtColor(img, colorTransformation)
    if channel != None:
        return Lpb(img[:,:,channel], isGray=True)
    return Lpb(img)

def get_LPB_Rcomp(aFileName):
    return get_LPB(aFileName, cv2.COLOR_BGR2RGB, 0)

def get_LPB_Gcomp(aFileName):
    return get_LPB(aFileName, cv2.COLOR_BGR2RGB, 1)

def get_LPB_Bcomp(aFileName):
    return get_LPB(aFileName, cv2.COLOR_BGR2RGB, 2)

def generate_LPB_on_RGB_separated(processBF = True, processM = True):
    # R color channel processing
    print('Generating: LPB for R color channel...')
    _TARGET_SUFFIX_ = 'lpb-Rcomp-cropped'
    if processBF:
        _BF_TARGET_PATH_ = f'data/{_DATABASE_}-Processed/bona-fide-{_TARGET_SUFFIX_}/'
        apply_transformation(bf_filenames, get_LPB_Rcomp, _BF_TARGET_PATH_)
    if processM:
        _MORPH_TARGET_PATH = f'data/{_DATABASE_}-Processed/morphed-{_MORPHING_ALGO_}-{_TARGET_SUFFIX_}/'
        apply_transformation(m_filenames, get_LPB_Rcomp, _MORPH_TARGET_PATH)  

    #G color channel processing
    print('Generating: LPB for G color channel...')
    _TARGET_SUFFIX_ = 'lpb-Gcomp-cropped'
    if processBF:
        _BF_TARGET_PATH_ = f'data/{_DATABASE_}-Processed/bona-fide-{_TARGET_SUFFIX_}/'
        apply_transformation(bf_filenames, get_LPB_Gcomp, _BF_TARGET_PATH_)
    if processM:
        _MORPH_TARGET_PATH = f'data/{_DATABASE_}-Processed/morphed-{_MORPHING_ALGO_}-{_TARGET_SUFFIX_}/'
        apply_transformation(m_filenames, get_LPB_Gcomp, _MORPH_TARGET_PATH)  

    #B color channel processing
    print('Generating: LPB for B color channel...')
    _TARGET_SUFFIX_ = 'lpb-Bcomp-cropped'
    if processBF:
        _BF_TARGET_PATH_ = f'data/{_DATABASE_}-Processed/bona-fide-{_TARGET_SUFFIX_}/'
        apply_transformation(bf_filenames, get_LPB_Bcomp, _BF_TARGET_PATH_)
    if processM:
        _MORPH_TARGET_PATH = f'data/{_DATABASE_}-Processed/morphed-{_MORPHING_ALGO_}-{_TARGET_SUFFIX_}/'
        apply_transformation(m_filenames, get_LPB_Bcomp, _MORPH_TARGET_PATH)

### MAIN to run

In [7]:
# load pics
bf_filenames = [_BF_TARGET_PATH_+f for f in listdir(_BF_TARGET_PATH_) if isfile(join(_BF_TARGET_PATH_, f))]
m_filenames = [_MORPH_TARGET_PATH+f for f in listdir(_MORPH_TARGET_PATH) if isfile(join(_MORPH_TARGET_PATH, f))]
print('Number of Bona Fide pictures: ', len(bf_filenames))
print('Number of Morphed pictures: ', len(m_filenames))

Number of Morphed pictures:  964


In [8]:
generate_LPB_on_RGB_separated()

Generating: LPB for R color channel...


100%|██████████| 964/964 [00:00<00:00, 107118.88it/s]


Generating: LPB for G color channel...


100%|██████████| 964/964 [00:00<00:00, 107002.65it/s]


Generating: LPB for B color channel...


100%|██████████| 964/964 [00:00<00:00, 85136.63it/s]


# THE END.